<a href="https://colab.research.google.com/github/Bigote93/Chatbot_Alura_Sprint1/blob/main/Chatbot_Inteligente_Alura_Sprint1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprint 01 - Chatbot - Alura Latam

### (*) Configuracion ambiente

In [88]:
# ELiminacion de las alertas del modelo
import warnings
warnings.filterwarnings("ignore")

In [89]:
!pip install spacy
!pip install transformers
!pip install jellyfish
!python -m spacy download es_core_news_md

2023-10-11 23:41:14.859207: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 33.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [56]:
# Instaalador de lenguajes y captadores de elementos
#Definiendo variables del proyecto:
import spacy

nlp = spacy.load('es_core_news_md')

In [57]:
#Recursos y librerias
import pandas as pd
import re, os, random, pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import spacy
import jellyfish

from transformers import BertForSequenceClassification
from transformers import BertTokenizer

import torch

#nltk.download('punkt')

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## (1) Lista de carga de verbos

In [59]:
#Instanciamos la lista de verbos
verbos_irregulares = []
#Direccion de lista de verbos
path_lista_verbos = '/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle'

#Comprobar el tamaño del archivo
tamaño_archivo = os.path.getsize(path_lista_verbos)

if tamaño_archivo == 0:
    print("El archivo está vacío.")
else:
    #Apertura de documentos
    fichero_listaverbos = open(path_lista_verbos, 'rb')

    #Cargar fichero
    verbos_irregulares = pickle.load(fichero_listaverbos)

In [60]:
#Instanciamos la lista de verbos
lista_verbos = []
#Direccion de lista de verbos
path_lista_verbos = '/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle'

#Comprobar el tamaño del archivo
tamaño_archivo = os.path.getsize(path_lista_verbos)

if tamaño_archivo == 0:
    print("El archivo está vacío.")
else:
    #Apertura de documentos
    fichero_listaverbos = open(path_lista_verbos, 'rb')

    #Cargar fichero
    lista_verbos = pickle.load(fichero_listaverbos)



### Incorporando **verbos_irregulares** que no estan presente en **lista_verbos**

### Solucionar verbos separados por caracter <_**'  '**_ >  

In [62]:
# Verbos de la lista_verbos que tienen auxiliares o estan separados por caracteres de espacio

for verbo in lista_verbos:
  if ' ' in verbo:
    print(verbo)

amaran o amasen
amaramos o amasemos
wasapearais o wasapeaseis
hemos wasapeado
has wasapeado
ha wasapeado
he wasapeado
wasapeara o wasapease
amaras o amases
wasapearan o wasapeasen
amarais o amaseis
amara o amase
ellas wasapeaban
wasapearamos o wasapeasemos
wasapearas o wasapeases
han wasapeado


In [63]:
# Agregamos el verbo "wasapear"

# Conjugaciones
wasapear_conj = [
    'wasapeo','wasapeas','wasapea','wasapean','wasapeamos','wasapeais',
    'wasapee','wasapeaste','wasapeo','wasapeamos','wasapeasteis','wasapearon',
    'wasapeare','wasapearas','wasapeara','wasapearemos','wasapeareis','wasapearan'
    ]


# Incorporacion a lista verbos_irregulares
for whatsap_conj in wasapear_conj:
  verbos_irregulares[whatsap_conj] = 'wasapear'

In [64]:
# Eliminar verbos a conjugados
for verbo in lista_verbos:
  if ' ' in verbo:
    lista_verbos.remove(verbo)

# Verificamos su eliminacion
for verbo in lista_verbos:
  if ' ' in verbo:
    print(verbo)

In [65]:
# Verbos de los verbos_irregulares que tienen auxiliares o estan separados por caracteres de espacio
for verbo in verbos_irregulares.keys():
  if ' ' in verbo:
    print(verbo)

for verbo in verbos_irregulares.values():
  if ' ' in verbo:
    print(verbo)

## (2) Tratamiento de datos

### Pronombres y adverbios

Apartado opcional a es_core_sm si existe algun error en el proceso de identificar palabras.

In [66]:
pronombres = [
    "yo","tu","el","ella","usted","nosotros","nosotras","vosotros",
    "vosotras","ellos","ellas","ustedes",
    "mi","tu","su","nuestro","nuestra","vuestro","vuestra",
    "su","este","esta","ese","esa","aquel","aquella","estos","estas",
    "esos","esas","aquellos","aquellas","que","cual","quien","cuyo","cuya",
    "donde","alguien","nadie","algun","alguna","ningun","ninguna","cualquiera",
    "nada","algo","todo","toda"]

adverbios_de_lugar = [
    "aqui", "ahi", "alli", "cerca", "lejos", "encima", "debajo",
    "delante", "detras", "adentro", "afuera", "arriba", "abajo",
    "fuera", "dentro", "aca", "alla", "donde"
    ]

adverbios_de_tiempo = [
    "ahora", "antes", "despues", "hoy", "manana", "tarde", "temprano",
    "siempre", "nunca", "ayer", "pronto", "luego", "todavia", "ya",
    "anteriormente", "actualmente", "frecuentemente", "raramente", "todavia",
    "tambien", "todavia", "tampoco", "ya", "hoy"
    ]

adverbios_de_modo = [
    "asi", "bien", "mal", "mejor", "peor", "lentamente",
    "rapidamente", "claramente", "directamente", "fuertemente",
    "suavemente", "cuidadosamente", "juntos", "separados",
    "adrede", "aposta", "deprisa", "despacio", "felizmente",
    "tristemente", "correctamente", "incorrectamente"
    ]

adverbios_de_cantidad = [
    "mucho", "poco", "bastante", "mas", "menos", "demasiado",
    "casi", "exactamente", "muy", "tan", "poco", "todo", "nada",
    "algo", "casi", "medio", "suficiente", "apenas", "bien", "mal"
    ]

adverbios_de_informacion = ["cuando", "donde", "como", "porque", "cuanto", "cual", "quien", "que"]

adverbios_de_afirmacion_y_negacion = ["si", "no", "tampoco", "así", "también", "jamás", "nunca", "sí", "certamente", "seguramente", "indudablemente"]

advervios = adverbios_de_lugar + adverbios_de_tiempo + adverbios_de_modo + adverbios_de_cantidad + adverbios_de_informacion + adverbios_de_afirmacion_y_negacion

regex_words = pronombres + advervios



In [67]:
# Verificar si existe un regex_words en ls lista verbos
elementos_a_eliminar = []
for elemento in lista_verbos:
  if elemento in regex_words:
    elementos_a_eliminar.append(elemento)

elementos_a_eliminar

['nosotros', 'ellos', 'ella', 'vosotros', 'ustedes', 'ellas']

In [68]:
# ELiminar pronombres de la lista_verbos

lista_verbos = [elemento for elemento in lista_verbos if elemento not in elementos_a_eliminar]

# Verificar si existen los elementos eliminados

elementos_a_eliminar = []
for elemento in lista_verbos:
  if elemento in regex_words:
    elementos_a_eliminar.append(elemento)

len((elementos_a_eliminar))


0

In [69]:
# Cantidad de verbos
print(f'Lista de verbos infinitivos: {len(lista_verbos)}')
print(f'Lista de verbos irregulares: {len(verbos_irregulares.keys())}')

Lista de verbos infinitivos: 1725
Lista de verbos irregulares: 131


### Raiz de los verbos

Función raiz que reciba una palabra y la compare con todas las palabras de la lista_verbos utilizando jaro_winkler , y que devuelva la palabra de lista_verbos con mayor similaridad a la palabra ingresada.

> Observación: Si la palabra encontrada, con mayor similaridad, no supera el radio de 0.93 entonces deberá regresar la palabra original.

In [70]:
def raiz_verbo(verbo_analizar):

  #Lista de verbos cercanos
  verbos_cercanos = []

  #Recorremos la lista de verbos
  for verbo in lista_verbos:
    #Condicionador de verbos y adjuntar a la lista
    if jellyfish.jaro_winkler_similarity(verbo,verbo_analizar) >= 0.93:
      # Creamoos un array de dos variables de la palabra y el valor de probabilidad cercania
      verbos_cercanos.append([jellyfish.jaro_winkler_similarity(verbo_analizar,verbo), verbo])

    else:
      verbo_cercano = verbo_analizar

  # Extraer el valor mas alto
  if len(verbos_cercanos)>0:
    verbo_cercano = max(verbos_cercanos, key=lambda x: x[0])[1]

  return verbo_cercano

#Aplicacion
print(raiz_verbo('wasapior'))

wasapior


### Tratamiento de texto

Función tratamiento_texto que reciba una frase de texto y devuelva la misma frase pero sin acentuaciones, todo en minúscula y sin espacios en blanco adicionales.

Ejemplo:

```
# Ejemplo visualizacion

tratamiento_texto('¡Buen día!, ¿Cómo está todo hoy?')

>> '¡buen dia!, ¿como esta todo hoy?'

```



In [71]:
def tratamiento_texto(texto):

  # Convertimos el texto a minúsculas.
  if texto is not None:
    texto_minusculas = texto.lower()
    # Eliminamos los caracteres con tildes.
    texto_tratado = texto_minusculas.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
    return texto_tratado

tratamiento_texto('Comisté')

'comiste'

### Terminacion de palabras

Función reemplazar_terminacion que reciba una palabra e identifique si la misma termina en alguna de las siguientes palabras: “es”, “me”, “as”, “te”, “ste”, si coincide, entonces que substituya esa terminación por la letra “r”.

In [72]:
def reemplazar_terminacion(palabra):

  terminaciones = ['ste','es','me','as','te',]
  #Verificar si tiene terminacion

  for terminacion in terminaciones:
    if palabra.endswith(terminacion):

      #Cambiar la terminacion de la palabra
      verbo_infinitivo = palabra[:-len(terminacion)]+'r'

      #Asignar la palabra modificada a la variable
      return verbo_infinitivo
    # En caso de no terminar con alguna terminacion, se devuelve la palabra ingresada
    else:
      return palabra

# Ejemplo de uso:
reemplazar_terminacion('comiste')


'comir'

### Normalizacion de textos

Crear una funcion que identifique la categoria de la palabra.

>Observacion: Antes de realizarlo, debemos aplicar la funcion de puntuacion y tildes para limpiar la oracion

> No usaremos el modelo nlp debido a que varios verbos no son reconocidos y son atribuidos a una posicion diferente. (1476 verbos mal clasificados)

Ejemplo: 'cosas' -> VERB


In [73]:

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz_verbo(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  return  ' '.join(tokens)


print(normalizar("Por favor, dime cómo se gusta un estas café, endulzarlo con azúcar y disfrutarlo en la terraza."))

favor decir como el gustar estar cafe endulzar azucar disfrutar terraza


## (3) Cargar base de documentos

### Cargar preguntas

In [74]:
# Generamos una ruta relativa para busqueda de elementos
direccion_relativa = '/content/drive/MyDrive/Chatbot'
dir_relativa_entrenamiento = direccion_relativa + '/dialogos'

# Barrido de documentos internos el folder
titulo_doc_list = [documento for documento in os.listdir(dir_relativa_entrenamiento) if documento.endswith('.txt')]

# Creamos listas para almacenar contenidos
lista_dialogos, lista_dialogos_respuesta, lista_dialogos_tipo = [], [], []

#Recorremos los documentos para almacenar las oraciones
for name_doc in range(len(titulo_doc_list)):

  # Apertura de documentos con los titulos almacenados
  doc = open(
      dir_relativa_entrenamiento + '/' + titulo_doc_list[name_doc],
      'r',
      encoding = 'utf-8',
      errors = 'ignore'
      )

  # Separar en lineas por cada texto
  lineas = doc.read().split('\n')

  # Carga de lineas del documento abierto
  for index, linea in enumerate(lineas):



    if index % 2 != 0: # Par: Respuesta necesaria
      lista_dialogos_respuesta.append(
          tratamiento_texto(
              re.sub(
                  r"[^\w\s+\-*/]",
                  '',
                  linea
                  )
              )
          )

      # Almacenar el tipo de linea
      lista_dialogos_tipo.append(titulo_doc_list[name_doc].replace('.txt', ''))

    else: # Impar: Pregunta a bot
      lista_dialogos.append(
          tratamiento_texto(
              re.sub(
                  r"[^\w\s+\-*/]",
                  '',
                  linea
                  )
              ))
print(f'El tamaño de dialogos es: {len(lista_dialogos)}')
print(f'El tamaño de respuestas es: {len(lista_dialogos_respuesta)}')
print(f'El tamaño de tipo es: {len(lista_dialogos_tipo)}')

El tamaño de dialogos es: 1152
El tamaño de respuestas es: 1152
El tamaño de tipo es: 1152


In [75]:
# Creamos dataframe "df_dialogo"
datos_dialogo = {
    'dialogo': lista_dialogos,
    'respuesta': lista_dialogos_respuesta,
    'tipo': lista_dialogos_tipo,
    'interseccion': 0,
    'jaro_winkler': 0,
    'probabilidad': 0
}

#Instanciamos el dataframe
df_dialogo = pd.DataFrame(datos_dialogo)

#Eliminamos los registros que se han repetido
df_dialogo = df_dialogo.drop_duplicates(keep = 'first')

#Eliminamos el index del dataframe
df_dialogo.reset_index(drop=True, inplace=True)

df_dialogo['tipo'].unique()

array(['Aprendizaje', 'Edad', 'ElProfeAlejo', 'Error', 'Funcion',
       'Identidad', 'Nombre', 'Origen', 'Saludos', 'Sentimiento',
       'Usuario', 'Otros', 'Contacto', 'Continuacion', 'Despedida',
       'Agradecimiento'], dtype=object)

## (4) Buscar respuesta del Chatbot

### Comparacion de Textos

In [76]:
# Funcion para verificar si el usuario inicio un dialogo
def dialogo(pregunta_usuario, df):

  df_registros = df.copy()

  # Al igual que la normalizacion de preguntas, eliminamos caracteres como normalizados
  pregunta_user_tratada = re.sub(
      r"[^\w\s]",
      '',
      tratamiento_texto(pregunta_usuario)
      )

  # Crear un vectorizador y entrenarlo con tus datos
  vectorizador = TfidfVectorizer()
  tfidf_matrix = vectorizador.fit_transform(df_dialogo['dialogo'])

  # Funcion interseccion: Cantidad de palabras en comun
  def interseccion(texto_user, texto_comparar):

    #Transformamos a Set un string
    user_text = set(texto_user.split())
    df_text = set(texto_comparar.split())

    # Calculo de interseccion
    per_words_commun = len(user_text.intersection(df_text)) / len(user_text)

    return per_words_commun

  # Funcion similarity valor de similitud matematica
  def similarity(texto_user, texto_comparar):

    # Transformamos y obtenemos la union entre los registros
    text_user_vector = vectorizador.transform([texto_user])
    text_comparar_vector = vectorizador.transform([texto_comparar])

     # Calcular la similitud coseno entre el texto del usuario y el texto de comparación
    similarity_score = cosine_similarity(text_user_vector, text_comparar_vector)

    return similarity_score[0][0]


  # Recorremos cada fila para buscar cercania de pregunta con dialogo
  for idx, row in df_registros.iterrows():

    df_registros.at[idx, 'interseccion'] = interseccion(pregunta_user_tratada, row['dialogo'])
    df_registros.at[idx, 'jaro_winkler'] = jellyfish.jaro_winkler_similarity(pregunta_user_tratada,row['dialogo'])
    df_registros.at[idx, 'similarity'] = similarity(pregunta_user_tratada, row['dialogo'])
    df_registros.at[idx, 'probabilidad'] = max(df_registros.at[idx, 'interseccion'], df_registros.at[idx, 'similarity'], df_registros.at[idx, 'jaro_winkler'])

  return df_registros


In [77]:
# Ejemplo:

df_registros = dialogo('Me tengo que retirar a comer, gracias por tu ayuda', df_dialogo)

df_registros.sort_values(by = 'probabilidad', ascending=False)

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,similarity
1065,me alegro de haberte conocido hasta pronto,igualmente hasta pronto,Despedida,0.1,0.803102,0.803102,0.083425
958,me interesa curso de programacion,debes revisar el canal de elprofealejo en youtube,Otros,0.1,0.802756,0.802756,0.099812
1061,me alegra haberte podido ayudar hasta luego,gracias por todo hasta luego,Despedida,0.1,0.796589,0.796589,0.076288
1097,gracias por tu ayuda,no hay problema siempre estoy aqui para ayudarte,Agradecimiento,0.4,0.552041,0.756142,0.756142
1071,bueno me tengo que ir ya hasta la proxima,hasta la proxima,Despedida,0.3,0.741696,0.741696,0.329599
...,...,...,...,...,...,...,...
646,2 mas 2,no estoy autorizado a darte estos resultados,Otros,0.0,0.384354,0.384354,0.000000
952,xd lmao,jajaja,Otros,0.0,0.384354,0.384354,0.000000
492,saludos desde,hola espero que esten disfrutando de un dia ma...,Saludos,0.0,0.361329,0.361329,0.000000
744,dividir 5,no estoy autorizado a darte este tipo de infor...,Otros,0.0,0.353741,0.353741,0.000000


### Machine Learning

Clasificaremos ahora el modelo mediante un modelo de Transformers

In [78]:
!pip install transformers

In [79]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#### Normalizando dialogos

In [80]:
# Normalizando frases
label_encoder = LabelEncoder()

# Normalizar cada dialogo
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(normalizar)
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']

df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,palabras,tipo_num
0,aprendes cosas,aprendo de mi experiencia conversando con otro...,Aprendizaje,0,0,0,aprender cosa,1
1,como aprendiste a responder preguntas,aprendi a responder preguntas a traves de un e...,Aprendizaje,0,0,0,como aprender responder pregunta,1
2,puedes mejorar o actualizarte con el tiempo,si mi capacidad para responder preguntas y mi ...,Aprendizaje,0,0,0,poder mejorar actualizar tiempo,1
3,cual es tu nivel de inteligencia,soy un chatbot desarrollado para simular respu...,Aprendizaje,0,0,0,cual ser nivel inteligencia,1
4,puedes aprender de la interaccion con los usua...,como modelo de lenguaje no tengo la capacidad ...,Aprendizaje,0,0,0,poder aprender interaccion usuario,1
...,...,...,...,...,...,...,...,...
1141,esta muy bueno,muchas gracias estamos contentos de que te guste,Agradecimiento,0,0,0,mucho bueno,0
1142,te super agradezco por la ayuda,gracias a ti,Agradecimiento,0,0,0,tu super agradecer ayuda,0
1143,muchas gracias por compartir tus conocimientos...,gracias a ti,Agradecimiento,0,0,0,gracia compartir conocimiento estar realmente ...,0
1144,muy bien muy bien,es un placer saber que te gusta gracias,Agradecimiento,0,0,0,mucho bien,0


### (1) Naive bayes

Algunos recordatorios y observaciones:

_**CountVectorizer**_

> La instancia CountVectorizer genera la frase en un vector de manera que podemos pasar un array con cada una de las palabras y las vectorizara dela forma:

```
# palabra[0] = 'aprender cosas'
vectorizer.fit_transform(palabras)

palabra[0] -> [(0,331),(0,338),(0,543)]
```

> Esto genera que a cada frase se le genera un vectorizado con tantos elementos en tupla como palabras tenga separadas por caracteres de _**'
  '**_

_**MultinomialNB**_

> Corresponde a un clasificador probabilístico que se utiliza para clasificar datos de texto. Es un tipo de clasificador Naive Bayes, que se basa en la suposición de que las características de un documento son independientes entre sí.
El clasificador calcula la probabilidad de que un documento pertenezca a una determinada clase, dado el número de veces que aparecen cada uno de los términos en ese documento.



```
# Entrenamos el modelo con los datos de entrenamiento vectorizados de la variable independiente y los datos sin vectorizar de la variable dependiente. Esto debido a que buscamos identificar la variable de control dependiente, no es necesario hacer la transformacion a vectorizada

modelo_nb.fit(X_train, y_train)

# Generamos los datos de prediccion utilizando el modelo multibinomial

y_pred = modelo_nb.predict(X_test)


```

Y finalmente calculamos el accuracy del modelo utilizando los datos de test y prediccion identificados

```
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)
```






In [104]:
# Identificacion de variables para el modelo
independiente_var = df_dialogo['palabras']
dependiente_var = df_dialogo['tipo_num']

#Generacion del modelo de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(
    independiente_var,
    dependiente_var,
    test_size=0.3,
    random_state=272
    )

# Generamos el vectorizador
vectorizer = CountVectorizer()

# Transformamos los datos de entrenamiento y entrenamos el vectorizador
X_train= vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


# Usamos el modelo multibinomial de Naive Bayes
modelo_nb = MultinomialNB()

# Entrenamos el modelo con los datos vectorizados
modelo_nb.fit(X_train, y_train)

# Generamos la prediccion con los datos test que no han sido vectorizados con el modelo Multibinomial
y_pred = modelo_nb.predict(X_test)

# Generamos el nivel de efectividad de los datos de entrenamiento y los predichos vectorizados
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.6286549707602339


In [91]:
y_test

989      2
721     12
174      8
1004     3
388     13
        ..
439     13
860     12
108      7
466     13
325     11
Name: tipo_num, Length: 342, dtype: int64

#### Verificando accuracy por categoria

In [121]:
# Calcular la precisión por clase

df_pred_naive = pd.DataFrame(columns=['Clase','Accuracy'])
df_pred_naive

categorias, accuracyes = [],[]
unique_classes = df_dialogo['tipo_num'].unique()
for clase in unique_classes:

    # Barrimos por cada id de la clase numerica tipo y luego identificamos el valor string de tipo y almacenamos los unicos en la lists categorias
    categorias.append(
        df_dialogo[df_dialogo.tipo_num == clase]['tipo'].unique()[0],
    )

    # Barrimos cada dato de entrenamiento cuando sea de la clase y al igual que la prediccion, y calculamos la predicciones correctas por cada tipo de dato
    accuracyes.append(
        accuracy_score(y_test[y_test == clase], y_pred[y_test == clase])
    )


# Almacenamos en el dataframe los valores cruzados entre categoria y exito de prediccion

df_pred_naive['Clase'] = categorias
df_pred_naive['Accuracy'] = accuracyes

# Ordenamos el dataframe por mejor resutlado del modelo
df_pred_naive.sort_values(by='Accuracy', ascending=False)



,Clase,Accuracy
14,Despedida,0.928571
11,Otros,0.886598
8,Saludos,0.823529
10,Usuario,0.700000
13,Continuacion,0.700000
5,Identidad,0.650000
2,ElProfeAlejo,0.625000
1,Edad,0.571429
0,Aprendizaje,0.500000
15,Agradecimiento,0.470588


#### Procesamiento de input

In [125]:

# Procesando la nueva frase
frase = ' '.join(normalizar('como haces para aprender tan rapido?'))
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = modelo_nb.predict(nueva_frase_vect)


claves_categorias = df_pred_naive[['Clase']].to_dict(orient='records')
print(claves_categorias)


[{'Clase': 'Aprendizaje'}, {'Clase': 'Edad'}, {'Clase': 'ElProfeAlejo'}, {'Clase': 'Error'}, {'Clase': 'Funcion'}, {'Clase': 'Identidad'}, {'Clase': 'Nombre'}, {'Clase': 'Origen'}, {'Clase': 'Saludos'}, {'Clase': 'Sentimiento'}, {'Clase': 'Usuario'}, {'Clase': 'Otros'}, {'Clase': 'Contacto'}, {'Clase': 'Continuacion'}, {'Clase': 'Despedida'}, {'Clase': 'Agradecimiento'}]


### (2) Random Forest

Algunos recordatorios y observaciones:

_**RandomForestClassifier()**_

> RandomForestClassifier es un clasificador de aprendizaje automático que utiliza un conjunto de árboles de decisión para predecir la clase de una nueva instancia.

>> Los _**árboles de decisión**_ son modelos de aprendizaje automático que aprenden a clasificar instancias dividiendo el espacio de características en regiones cada vez más pequeñas, hasta que cada región solo contenga instancias de una clase.

>> Cada árbol de decisión se entrena en un _**subconjunto aleatorio de las características**_ y un subconjunto aleatorio de las instancias.

 Para predecir la clase de una nueva instancia, el _**RandomForestClassifier**_ calcula la predicción de cada árbol de decisión en el conjunto y luego _asigna la clase más popular entre las predicciones de los árboles._




```
# Instanciamos el modelo de RandomForestClassifier

modelo_rf = RandomForestClassifier(
  n_estimators = 500, # Cantidad de ramas de decision
  n_jobs=-1, # Cantidad de nucleos GPU
  random_state=123  # Semilla
  )

```
> _**n_jobs**_ : n_jobs es el número de núcleos de CPU que se utilizarán para entrenar el modelo

> _**n_estimators**_ = Número de árboles de decisión que se utilizan en el modelo. Un valor más alto de n_estimators generalmente conducirá a un modelo más preciso, ya que el modelo tendrá en cuenta más información de los datos de entrenamiento.



In [90]:

from sklearn.ensemble import RandomForestClassifier
modelo_rf = RandomForestClassifier(n_estimators = 500, n_jobs=-1,random_state=123)
modelo_rf.fit(X_train, y_train)

y_pred = modelo_rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)


Precisión: 0.7017543859649122


### (3) Transformers